<link rel="preconnect" href="https://fonts.gstatic.com">
<link href="https://fonts.googleapis.com/css2?family=Fira+Code&display=swap" rel="stylesheet">

### License

<p style="font-family: 'Fira Code', monospace; font-size: 1.2rem">
Copyright 2021 Xilinx, Inc.<br><br>
Licensed under the Apache License, Version 2.0 (the "License");<br>
you may not use this file except in compliance with the License.<br><br>
You may obtain a copy of the License at <a href="http://www.apache.org/licenses/LICENSE-2.0"?>http://www.apache.org/licenses/LICENSE-2.0</a><br><br>
Unless required by applicable law or agreed to in writing, software<br>
distributed under the License is distributed on an "AS IS" BASIS,<br>
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.<br>
See the License for the specific language governing permissions and<br>
limitations under the License.<br>
</p>


# ChipScoPy ILA and VIO Example


<img src="../img/api_overview.png" width="500" align="left">

## Description
This example demonstrates how to program and communicate with ILA (Integrated Logic Analyzer) and
VIO (Virtual IO) cores using the ChipScoPy Python API.


## Requirements
- Local or remote Xilinx Versal board, such as a VCK190
- Xilinx hw_server 2021.1 or greater
- Xilinx cs_server 2021.1 or greater
- Python 3.8 or greater installed
- ChipScoPy 2021.1 or greater installed
- Jupyter notebook support installed - Please do so, using the command `pip install chipscopy[jupyter]`

## 1 - Initialization: Imports and File Paths

After this step,
- Required functions and classes are imported
- URL paths are set correctly
- File paths to example files are set correctly

In [1]:
import sys
import os
from chipscopy import get_design_files
from chipscopy import create_session, report_versions
from chipscopy.api.ila import get_waveform_data, export_waveform

In [2]:
# Make sure to start the hw_server and cs_server prior to running.
# Specify locations of the running hw_server and cs_server below.
# The default is localhost - but can be other locations on the network.
CS_URL = os.getenv("CS_SERVER_URL", "TCP:localhost:3042")
HW_URL = os.getenv("HW_SERVER_URL", "TCP:localhost:3121")

# The get_design_files() function tries to find the PDI and LTX files. In non-standard
# configurations, you can put the path for PROGRAMMING_FILE and PROBES_FILE below.
design_files = get_design_files("vck190/production/chipscopy_ced")

PROGRAMMING_FILE = design_files.programming_file
PROBES_FILE = design_files.probes_file

print(f"HW_URL: {HW_URL}")
print(f"CS_URL: {CS_URL}")
print(f"PROGRAMMING_FILE: {PROGRAMMING_FILE}")
print(f"PROBES_FILE:{PROBES_FILE}")

HW_URL: TCP:localhost:3121
CS_URL: TCP:localhost:3042
PROGRAMMING_FILE: C:\wrk\chipscopy\chipscopy\examples\designs\vck190\production\chipscopy_ced\chipscopy_ex_wrapper.pdi
PROBES_FILE:C:\wrk\chipscopy\chipscopy\examples\designs\vck190\production\chipscopy_ced\chipscopy_ex_wrapper.ltx


## 2 - Create a session and connect to the hw_server and cs_server

The session is a container that keeps track of devices and debug cores.
After this step,
- Session is initialized and connected to server(s)
- Versions are detected and reported to stdout

In [3]:
session = create_session(cs_server_url=CS_URL, hw_server_url=HW_URL)
report_versions(session)

## 3 - Program the device with the example design

After this step,
- Device is programmed with the example programming file

In [4]:
# Typical case - one device on the board - get it.
device = session.devices.filter_by(family="versal").get()
device.program(PROGRAMMING_FILE)

Output()

## 4 - Discover Debug Cores

Debug core discovery initializes the chipscope server debug cores. This brings debug cores in the chipscope server online.

After this step,

- The cs_server is initialized and ready for use
- ILA and VIO core instances in the device are reported

In [5]:
device.discover_and_setup_cores(ltx_file=PROBES_FILE)
print(f"Debug cores setup and ready for use.")

Debug cores setup and ready for use.


In [6]:
# Print out the ILA core instance UUIDs and instance names
ila_cores = device.ila_cores
for index, ila_core in enumerate(ila_cores):
    print(f"{index} - {ila_core.core_info.uuid}   {ila_core.name}")

0 - 24AB5B9D35BF59B5A72FBDB8F1AF8263   chipscopy_ex_i/counters/ila_fast_counter_0
1 - 55CD0B15530552099FD8BABE573DB8BC   chipscopy_ex_i/counters/ila_slow_counter_0
2 - 7387EB729C5B5216B1B700DC2CB2889C   chipscopy_ex_i/noc_tg_bc/noc_bc_axis_ila_0


In [7]:
# Print out the VIO core instance UUIDs and instance names
vio_cores = device.vio_cores
for index, vio_core in enumerate(vio_cores):
    print(f"{index} - {vio_core.core_info.uuid}   {vio_core.name}")

0 - 3C824110DF5952A79F699CF4B5C189AE   chipscopy_ex_i/counters/vio_fast_counter_0
1 - 4A8AF42C6C71542D968D58CE5AF56CBC   chipscopy_ex_i/counters/vio_slow_counter_0
2 - EE72A80322AC596E9265DB49D420CB9C   chipscopy_ex_i/noc_tg_bc/axis_vio_0
3 - F83580982EDC5FD2B1A441514DA7CD37   chipscopy_ex_i/noc_tg_bc/noc_tg/inst/u_top_axi_mst/u_tg_top/gen_aximm_reg_space.u_reg_space/gen_vio_status_monitor.u_vio_status_monitor


## 5 - VIO Control and ILA Capture

ILA and VIO are two important building blocks for debugging applications in hardware.
This example design design shows how to control IP using a VIO core and capture results with ILA.

In this Design,
- A VIO core controls the counter (reset, up/down, ce, load)
- An ILA core captures the counter values


<img src="img/capture_data.png" width="400" align="left">

In [8]:
# Grab the two cores we are interested in for the demonstration
# As shown above, a counter is connected to the ILA core.
# The VIO core controls the counter.

ila = device.ila_cores.get(name="chipscopy_ex_i/counters/ila_slow_counter_0")
vio = device.vio_cores.get(name="chipscopy_ex_i/counters/vio_slow_counter_0")

print(f"Using ILA: {ila.core_info.uuid}  {ila.name}")
print(f"Using VIO: {vio.core_info.uuid}  {vio.name}")

Using ILA: 55CD0B15530552099FD8BABE573DB8BC  chipscopy_ex_i/counters/ila_slow_counter_0
Using VIO: 4A8AF42C6C71542D968D58CE5AF56CBC  chipscopy_ex_i/counters/vio_slow_counter_0


### 5a - Configure the counter using VIO output probes

<img src="img/vio_control_counter.png" width="300" align="left">

In [9]:
# Print all the VIO port and probe names. This is convenient to know which probes are connected to
# VIO ports. Also verifies probe names to pass to other functions.

print("VIO Port <---> Probe mapping:")
for probe in vio.probes:
    if probe.direction == "in":
        print(f"{probe.port_name} <-- {probe.probe_name}")
    else:
        print(f"{probe.port_name} --> {probe.probe_name}")

VIO Port <---> Probe mapping:
probe_in0 <-- chipscopy_ex_i/counters/slow_counter_0_Q
probe_out0 --> chipscopy_ex_i/counters/slow_counter_0_CE
probe_out1 --> chipscopy_ex_i/counters/slow_counter_0_SCLR
probe_out2 --> chipscopy_ex_i/counters/slow_counter_0_UP
probe_out3 --> chipscopy_ex_i/counters/slow_counter_0_LOAD
probe_out4 --> chipscopy_ex_i/counters/slow_counter_0_L


In [10]:
# Set up the VIO core to enable counting up from 0
#
vio.reset_vio()
vio.write_probes(
    {
        "chipscopy_ex_i/counters/slow_counter_0_SCLR": 0,
        "chipscopy_ex_i/counters/slow_counter_0_L": 0x00000000,
        "chipscopy_ex_i/counters/slow_counter_0_LOAD": 0,
        "chipscopy_ex_i/counters/slow_counter_0_UP": 1,
        "chipscopy_ex_i/counters/slow_counter_0_CE": 1,
    }
)
print("Counter is now free-running and counting up")

Counter is now free-running and counting up


### 5b - Capture and display free-running counter using the ILA core

<img src="img/free_running_counter.png" width="350" align="left">

In [11]:
# Trigger ILA on the free running counter. Trigger set to the first time we see 0s in low 16-bits.
# This will show the counter is free running, and counting up

ila.reset_probes()
ila.set_probe_trigger_value("chipscopy_ex_i/counters/slow_counter_0_Q_1", ["==", "0xXXXX_0000"])
ila.run_basic_trigger(window_count=1, window_size=32, trigger_position=16)
print("ILA is running - looking for trigger")

ILA is running - looking for trigger


In [12]:
# Wait for the ILA trigger with upload.
# Then print the captured ILA samples and mark the trigger position.

ila.wait_till_done(max_wait_minutes=0.1)
upload_successful = ila.upload()
if upload_successful:
    samples = get_waveform_data(
        ila.waveform,
        ["chipscopy_ex_i/counters/slow_counter_0_Q_1"],
        include_trigger=True,
        include_sample_info=True,
    )
    for trigger, sample_index, window_index, window_sample_index, value in zip(*samples.values()):
        trigger = "<-- Trigger" if trigger else ""
        print(
            f"Window:{window_index}  Window Sample:{window_sample_index}  {value:10}  0x{value:08X} {trigger}"
        )
else:
    print("Failed to upload ILA data from core")

Window:0  Window Sample:0  1935605744  0x735EFFF0 
Window:0  Window Sample:1  1935605745  0x735EFFF1 
Window:0  Window Sample:2  1935605746  0x735EFFF2 
Window:0  Window Sample:3  1935605747  0x735EFFF3 
Window:0  Window Sample:4  1935605748  0x735EFFF4 
Window:0  Window Sample:5  1935605749  0x735EFFF5 
Window:0  Window Sample:6  1935605750  0x735EFFF6 
Window:0  Window Sample:7  1935605751  0x735EFFF7 
Window:0  Window Sample:8  1935605752  0x735EFFF8 
Window:0  Window Sample:9  1935605753  0x735EFFF9 
Window:0  Window Sample:10  1935605754  0x735EFFFA 
Window:0  Window Sample:11  1935605755  0x735EFFFB 
Window:0  Window Sample:12  1935605756  0x735EFFFC 
Window:0  Window Sample:13  1935605757  0x735EFFFD 
Window:0  Window Sample:14  1935605758  0x735EFFFE 
Window:0  Window Sample:15  1935605759  0x735EFFFF 
Window:0  Window Sample:16  1935605760  0x735F0000 <-- Trigger
Window:0  Window Sample:17  1935605761  0x735F0001 
Window:0  Window Sample:18  1935605762  0x735F0002 
Window:0  W

### 5c - Trigger ILA using VIO Up/Down virtual switch

This step demonstrates how VIO and ILA can be combined to form powerful debug building blocks.

ILA is set to trigger when UP/DOWN counter signal edge rises or falls.
VIO drives the UP/DOWN counter control signal to 0 causing the counter to count down.
The signal transition causes ILA to trigger and capture data.

After this step,
- VIO drives counter to count from UP to DOWN
- ILA triggers on the UP to DOWN signal transition
- Waveform uploaded with the up/down trigger sample in the center of captured data


<img src="img/edge_trigger.png" width="550" align="left">

In [13]:
# Set ILA core to capture on a transition of the UP/DOWN toggle switch
# Once transition happens, trigger in the middle of the buffer.

ila.reset_probes()
ila.set_probe_trigger_value("chipscopy_ex_i/counters/slow_counter_0_UP_1", ["==", "B"])
ila.run_basic_trigger(window_count=1, window_size=32, trigger_position=16)

print("ILA is running - looking for trigger")

ILA is running - looking for trigger


In [14]:
# VIO: Turn counter up/down switch to DOWN position.
# This will cause the running ILA to trigger on the transition edge from up to down.

vio.write_probes({"chipscopy_ex_i/counters/slow_counter_0_UP": 0})

print("VIO changed up/down counter to count down")

VIO changed up/down counter to count down


In [15]:
# Print the captured ILA samples and mark the trigger position.
# Note that counter counts down after the trigger mark.

ila.wait_till_done(max_wait_minutes=0.1)
upload_successful = ila.upload()
if upload_successful:
    samples = get_waveform_data(
        ila.waveform,
        ["chipscopy_ex_i/counters/slow_counter_0_Q_1"],
        include_trigger=True,
        include_sample_info=True,
    )
    for trigger, sample_index, window_index, window_sample_index, value in zip(*samples.values()):
        trigger = "<-- Trigger" if trigger else ""
        print(
            f"Window:{window_index}  Window Sample:{window_sample_index}  {value:10}  0x{value:08X} {trigger}"
        )
else:
    print("Failed to upload ILA data from core")

Window:0  Window Sample:0  2060883338  0x7AD6958A 
Window:0  Window Sample:1  2060883339  0x7AD6958B 
Window:0  Window Sample:2  2060883340  0x7AD6958C 
Window:0  Window Sample:3  2060883341  0x7AD6958D 
Window:0  Window Sample:4  2060883342  0x7AD6958E 
Window:0  Window Sample:5  2060883343  0x7AD6958F 
Window:0  Window Sample:6  2060883344  0x7AD69590 
Window:0  Window Sample:7  2060883345  0x7AD69591 
Window:0  Window Sample:8  2060883346  0x7AD69592 
Window:0  Window Sample:9  2060883347  0x7AD69593 
Window:0  Window Sample:10  2060883348  0x7AD69594 
Window:0  Window Sample:11  2060883349  0x7AD69595 
Window:0  Window Sample:12  2060883350  0x7AD69596 
Window:0  Window Sample:13  2060883351  0x7AD69597 
Window:0  Window Sample:14  2060883352  0x7AD69598 
Window:0  Window Sample:15  2060883353  0x7AD69599 
Window:0  Window Sample:16  2060883354  0x7AD6959A <-- Trigger
Window:0  Window Sample:17  2060883353  0x7AD69599 
Window:0  Window Sample:18  2060883352  0x7AD69598 
Window:0  W

## 6 - Waveform Export - VCD (or CSV)

 Demonstrate how to export waveform data to a VCD file for visualizing waveform in other tools. 
 
 Export includes complete waveform with probe, _TRIGGER, and _WINDOW.
 
 - For CSV export, substitute "CSV" for "VCD" argument.
 - To export to a file, substitute the filename for 'sys.stdout'

In [16]:
if upload_successful:
    export_waveform(ila.waveform, "VCD", sys.stdout)

$date
        2021-06-30 20:08:37
$end
$version
        ChipScoPy Version 2021.1.1623743086
$end
$timescale
        1ps
$end
$scope module dut $end
$var reg 1 # chipscopy_ex_i/counters/slow_counter_0_CE_1  $end
$var reg 1 $ chipscopy_ex_i/counters/slow_counter_0_SCLR_1  $end
$var reg 1 % chipscopy_ex_i/counters/slow_counter_0_UP_1  $end
$var reg 1 & chipscopy_ex_i/counters/slow_counter_0_LOAD_1  $end
$var reg 32 ' chipscopy_ex_i/counters/slow_counter_0_L_1 [31:0] $end
$var reg 32 ( chipscopy_ex_i/counters/slow_counter_0_Q_1 [31:0] $end
$var reg 1 ) chipscopy_ex_i/counters/ila_slow_counter_0_TRIG_OUT_trig  $end
$var reg 1 * chipscopy_ex_i/counters/ila_fast_counter_0_TRIG_IN_ack  $end
$var reg 1 + chipscopy_ex_i/counters/ila_fast_counter_0_TRIG_OUT_trig  $end
$var reg 1 , chipscopy_ex_i/counters/ila_slow_counter_0_TRIG_IN_ack  $end
$var reg 16 - chipscopy_ex_i/counters/slow_sine_Dout [15:0] $end
$var reg 16 . chipscopy_ex_i/counters/slow_cosine_Dout [15:0] $end
$var reg 1 ! _TRIGGER $end